In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

import setting

path_dict = path(setting)

In [ ]:
training__feature_x_sample = pd.read_table(
    path_dict['feature_x_sample_file_path'],
    index_col=0,
)

gps_map = ccal.load_gps_map(path_dict['gps_map_file_path'])

training__w = pd.DataFrame(
    gps_map.wt.T,
    index=gps_map.wt_elements,
    columns=gps_map.nodes,
)

training__feature_x_sample

In [ ]:
testing__feature_x_sample_file_path = '../data/testing__feature_x_sample.tsv'

testing__feature_x_sample = ccal.read_and_process_feature_x_sample(
    testing__feature_x_sample_file_path,
    nanize=0,
    drop_na_axis=setting.DROP_NA_AXIS,
    max_na=setting.MAX_NA,
    log_base=setting.LOG_BASE,
    normalization_axis=setting.NORMALIZATION_AXIS,
    normalization_method=setting.NORMALIZATION_METHOD,
    max_plot_n=setting.MAX_PLOT_N,
    plot=False,
)

testing__feature_x_sample = testing__feature_x_sample.loc[training__w.index.str.lstrip('(-+) ').unique()]

testing__feature_x_sample

In [ ]:
testing__signal_matrix = np.full(
    (
        training__w.shape[0],
        testing__feature_x_sample.shape[1],
    ),
    np.nan,
)

training__feature_skew_t_pdf_fit_parameter = pd.read_table(
    path_dict['feature_skew_t_pdf_fit_parameter_file_path'],
    index_col=0,
)

n = training__w.shape[0]

n_per_print =  n // 10

for i, training__w_feature_sign_w_feature in enumerate(training__w.index):
    
    if i % n_per_print == 0:
        
        print('{}/{} ...'.format(
            i,
            n,
        ))
    
    training__w_feature_sign, training__w_feature = training__w_feature_sign_w_feature.split()
    
    training__location, training__scale, training__degree_of_freedom, training__shape = training__feature_skew_t_pdf_fit_parameter.loc[training__w_feature, [
        'Location',
        'Scale',
        'Degree of Freedom',
        'Shape',
    ]]

    training__context_dict = ccal.compute_context(
        training__feature_x_sample.loc[training__w_feature].values,
        location=training__location,
        scale=training__scale,
        degree_of_freedom=training__degree_of_freedom,
        shape=training__shape,
        scale_with_kl=setting.SCALE_WITH_KL,
    )
    
    training__grid = training__context_dict['grid']

    training__context_indices = training__context_dict['context_indices']
    
    if training__w_feature_sign == '(-)':

        training__context_indices = -training__context_indices.clip(max=0)
    
    elif training__w_feature_sign == '(+)':
        
        training__context_indices = training__context_indices.clip(min=0)

    testing__feature_values = testing__feature_x_sample.loc[training__w_feature].values
    
#     training__context_indices = ccal.normalize_nd_array(
#         training__context_indices.clip(min=0),
#         None,
#         '0-1',
#     )

    not_nan = ~np.isnan(testing__feature_values)

    testing__signal_matrix[i][not_nan] = training__context_indices[[np.absolute(training__grid - value).argmin() for value in testing__feature_values[not_nan]]]

testing__signal_matrix = pd.DataFrame(
    testing__signal_matrix, 
    index=training__w.index,
    columns=testing__feature_x_sample.columns,
)

for sign_feature in np.random.choice(
    testing__signal_matrix.index,
    size=8,
    replace=False,
):

    feature = sign_feature.split()[1]
    
    testing__feature_values = testing__feature_x_sample.loc[feature].sort_values()
    
    testing__feature_signals = testing__signal_matrix.loc[sign_feature, testing__feature_values.index]

    ccal.plot_context(
        training__feature_x_sample.loc[feature],
        scale_with_kl=setting.SCALE_WITH_KL,
        title=sign_feature,
    )

    ccal.plot_points(
        ('Look Up Context', ),
        (testing__feature_values, ),
        (testing__feature_signals, ),
        texts=(testing__feature_values.index, ),
    )

In [ ]:
pull_power = 1

element_marker_size = 16

In [ ]:
testing__a = testing__signal_matrix.dropna(how='all')

testing__h = ccal.solve_for_nmf_h(
    testing__a,
    training__w.loc[testing__a.index],
    method='nnls',
)

title = 'Testing'

testing__h_element_states = gps_map.predict(
    'h',
    testing__h,
    pull_power=pull_power,
    title=title,
    element_marker_size=element_marker_size,
)

ccal.plot_heat_map(
    testing__h,
    normalization_axis=0,
    normalization_method='-0-',
    column_annotation=testing__h_element_states,
    cluster_axis=1,
    title=title,
    xaxis_title=testing__h.columns.name,
    yaxis_title=testing__h.index.name,
)

In [ ]:
feature_dicts = make_feature_dicts()

features_to_highlight = make_features_to_highlight()

In [ ]:
for feature_group, features in feature_group_features.items():
    
    df = feature_dicts[feature_group]['df']
    
    df = df[df.columns & testing__h.columns]
    
    annotation_type = feature_dicts[feature_group]['data_type']
    
    for feature in features:

        if feature in df.index:
            
            testing__h_element_states = gps_map.predict(
                'h',
                testing__h,
                pull_power=pull_power,
                annotation_x_element=df.loc[feature].to_frame().T,
                annotation_std_maxs=(3, ),
                annotation_types=(annotation_type, ),
                title='{} {}'.format(
                    feature_group,
                    feature,
                ),
                element_marker_size=element_marker_size,
            )